<a href="https://colab.research.google.com/github/praiz13/group-47_week-6/blob/main/week_6_group_47.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q kaggle tensorflow numpy matplotlib opencv-python
from google.colab import files
import os
import zipfile

# Upload your Kaggle API key (download from Kaggle Account Settings)
files.upload()  # Upload 'kaggle.json'

# Move the key to the correct directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d alistairking/recyclable-and-household-waste-classification

# Unzip the dataset
with zipfile.ZipFile('recyclable-and-household-waste-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('waste_data')
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Define image dimensions
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32

# Load dataset
data_dir = 'waste_data'

# Preprocess data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical'  # Binary classification (Recyclable vs. Household)
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Normalize pixel values
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Define a lightweight CNN model
model = models.Sequential([
    layers.Conv2D(16, 3, activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output (0=Household, 1=Recyclable)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2
)

# Evaluate accuracy
val_loss, val_acc = model.evaluate(val_ds)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

# Convert model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('waste_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TFLite!")

import cv2

# Load a test image (replace with your own image)
test_image = cv2.imread('plastic-bottle-blue-background.jpg')  # Example
test_image = cv2.resize(test_image, (IMG_HEIGHT, IMG_WIDTH))
test_image = np.expand_dims(test_image, axis=0) / 255.0  # Normalize

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path='waste_classifier.tflite')
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run inference
interpreter.set_tensor(input_details[0]['index'], test_image.astype(np.float32))
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

# Interpret prediction
prediction = "Recyclable" if output[0][0] > 0.5 else "Household"
print(f"Predicted Class: {prediction} (Confidence: {output[0][0]:.2f})")



Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/alistairking/recyclable-and-household-waste-classification
License(s): MIT
 95% 871M/920M [00:04<00:00, 91.4MB/s]
100% 920M/920M [00:04<00:00, 213MB/s] 
Found 15000 files belonging to 1 classes.
Using 12000 files for training.
Found 15000 files belonging to 1 classes.
Using 3000 files for validation.
Epoch 1/2


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


375/375 ━━━━━━━━━━━━━━━━━━━━ 206s 544ms/step - accuracy: 0.9962 - loss: 0.0124 - val_accuracy: 1.0000 - val_loss: 1.2206e-15
Epoch 2/2
375/375 ━━━━━━━━━━━━━━━━━━━━ 254s 521ms/step - accuracy: 1.0000 - loss: 4.5263e-11 - val_accuracy: 1.0000 - val_loss: 1.2173e-15
94/94 ━━━━━━━━━━━━━━━━━━━━ 18s 188ms/step - accuracy: 1.0000 - loss: 6.3806e-16
Validation Accuracy: 100.00%
Saved artifact at '/tmp/tmpkfvsp2ip'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  133106010976336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133106010978832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133106010978064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133106010979600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133106010979024: TensorSpec(shape=(), dtype=tf.resource, name